# Scraping de partidas de competições

## Bibliotecas

In [6]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from fake_useragent import UserAgent

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import os

## Header

In [2]:
ua = UserAgent()

headers = {"User-Agent": ua.random}

## Definindo classes

### Classe Partida

In [3]:
class PartidaCatch:

    def __init__(self):
        options = Options()
        options.add_argument("--headless")

        self._driver = webdriver.Chrome(options=options)
    
    def get_partida(self, link_estatistica_partida):
        self._driver.get(link_estatistica_partida)

        self._driver.implicitly_wait(1)
        
        return Partida(self._driver.page_source)

    def __del__(self):
        self._driver.quit()

class Partida:

    def __init__(self, pageHtml):

        pageSoup = BeautifulSoup(pageHtml, 'html.parser')

        self._carrega_partida(pageSoup)


    def _carrega_partida(self, pageSoup):
        self._estatisticas = \
        {
            'capacidade_estadio': 0,
            'publico_estadio': 0,
            'time_casa': '',
            'gols_casa': 0,
            'posse_casa': 0,
            'tentativas_casa': 0,
            'chutes_fora_casa': 0,
            'defesa_casa': 0,
            'escanteios_casa': 0,
            'cobrancas_falta_casa': 0,
            'faltas_cometidas_casa': 0,
            'impedimentos_casa': 0,
            'time_fora': '',
            'gols_fora': 0,
            'posse_fora': 0,
            'tentativas_fora': 0,
            'chutes_fora_fora': 0,
            'defesa_fora': 0,
            'escanteios_fora': 0,
            'cobrancas_falta_fora': 0,
            'faltas_cometidas_fora': 0,
            'impedimentos_fora': 0,
        }

        cap_table = pageSoup.find_all('table', {"class":"profilheader"})[0]

        self._estatisticas['publico_estadio'], self._estatisticas['capacidade_estadio'] = list(map(lambda x: x.text.strip(), cap_table.find_all('td')[1:3]))

        casa = pageSoup.find_all('div', {"class": "sb-team sb-heim"})[0]
        fora = pageSoup.find_all('div', {"class": "sb-team sb-gast"})[0]

        self._estatisticas['time_casa'] = casa.find_all('a')[1].text.strip()
        self._estatisticas['time_fora'] =  fora.find_all('a')[1].text.strip()

        gols = pageSoup.find_all('div', {"class": "sb-endstand"})
        self._estatisticas['gols_casa'], self._estatisticas['gols_fora'] = gols[0].text.strip().split(sep='(')[0].split(sep=':')

        posse = pageSoup.find_all('tspan')

        self._estatisticas['posse_casa'], self._estatisticas['posse_fora'] = [posse[2].text, posse[1].text]

        todas__estatisticas = [x.text for x in pageSoup.find_all('div', {"class": "sb-statistik-zahl"})]
        
        self._estatisticas['tentativas_casa'], self._estatisticas['tentativas_fora'] = todas__estatisticas[0:2]

        self._estatisticas['chutes_fora_casa'], self._estatisticas['chutes_fora_fora'] = todas__estatisticas[2:4]

        self._estatisticas['defesa_casa'], self._estatisticas['defesa_fora'] = todas__estatisticas[4:6]

        self._estatisticas['escanteios_casa'], self._estatisticas['escanteios_fora'] = todas__estatisticas[6:8]

        self._estatisticas['cobrancas_falta_casa'], self._estatisticas['cobrancas_falta_fora'] = todas__estatisticas[8:10]

        self._estatisticas['faltas_cometidas_casa'], self._estatisticas['faltas_cometidas_fora'] = todas__estatisticas[10:12]

        self._estatisticas['impedimentos_casa'], self._estatisticas['impedimentos_fora'] = todas__estatisticas[12:14]

    def get_partida_df(self):
        return pd.DataFrame({key: [self._estatisticas[key]] for key in self._estatisticas})
    
    def get_partida_est(self):
        return self._estatisticas

### Classe Liga

In [4]:
class Liga:
    _competicao = \
    {
        'lbr': (2024, 1996, 'https://www.transfermarkt.com.br/campeonato-brasileiro-serie-a/gesamtspielplan/wettbewerb/BRA1?saison_id='),
        'pl': (2023, 1992, 'https://www.transfermarkt.com.br/premier-league/gesamtspielplan/wettbewerb/GB1?saison_id=')
    }

    def get_competicoes_desc(self):
        return {'lbr': 'Campeonato Brasileiro', 'pl': 'Premier League'}
    
    def get_competicoes_temporadas(self):
        return {'lbr': (self._competicao['lbr'][0], self._competicao['lbr'][1]),
                'pl': (self._competicao['pl'][0], self._competicao['pl'][1])}

    def partidas_estatistica_links(self, comp_tag, temporada):
        # Key = Rodada, Value = links das partida na rodada
        links = {}

        season_url = self._competicao[comp_tag][2] + str(int(temporada) - 1)

        res = requests.get(url=season_url, headers=headers)

        if res.status_code != 200:
            raise Exception('Erro na página')
        
        pageSoup = BeautifulSoup(res.content, 'html.parser')

        rodadas = pageSoup.find_all('div', {"class": "large-6 columns"})

        for rodada in rodadas:
            rodada_atual = rodada.find_all('div', {"class": "content-box-headline"})[0].text.strip()
            
            links[rodada_atual] = list(map(lambda x: x['href'].strip(), rodada.find_all('a', {"class": "ergebnis-link"})))

            links[rodada_atual] = [f'https://www.transfermarkt.com.br{x.replace("/index/", "/statistik/")}' for x in links[rodada_atual]]

        return links

### Classe Copa

In [ ]:
class Copa:
    _competicao = \
    {
        'cbr': (2024, 2020, 'https://www.transfermarkt.com.br/copa-do-brasil/startseite/pokalwettbewerb/BRC?saison_id='),
        'la': (2023, 2015, 'https://www.transfermarkt.com.br/copa-libertadores/startseite/pokalwettbewerb/CLI?saison_id='),
        'cl': (2024, 2000, 'https://www.transfermarkt.com.br/liga-dos-campeoes/startseite/pokalwettbewerb/CL?saison_id=')
    }

    def get_competicoes_desc(self):
        return {'cbr': 'Copa do Brasil', 'la': 'Libertadores da América', 'cl': 'Champions League'}
    
    def get_competicoes_temporadas(self):
        return {'cbr': (self._competicao['cbr'][0], self._competicao['cbr'][1]),
                'la': (self._competicao['la'][0], self._competicao['la'][1]),
                'cl': (self._competicao['cl'][0], self._competicao['cl'][1])}
    
    def partidas_estatistica_links(self, comp_tag, temporada):
        # Key = Fase, Value = links das partida na Fase
        links = {}

        season_url = self._competicao[comp_tag][2] + str(int(temporada) - 1)

        res = requests.get(url=season_url, headers=headers)

        if res.status_code != 200:
            raise Exception('Erro na página')
        
        pageSoup = BeautifulSoup(res.content, 'html.parser')

        tabela = pageSoup.find_all('tbody')[1]

        infos_partidas = tabela.find_all('tr', class_=['rundenzeile', 'begegnungZeile'])

        fase = ''
        for item in infos_partidas:
            if item['class'][0] == 'rundenzeile':
                fase = item.find_all('a')[0].text.strip()
                links[fase] = []
            elif item['class'][0] == 'begegnungZeile':
                partida_link = item.find_all('a', {'title': 'Colocar informaçãoes online?'})[0]['href'].strip().replace("/index/", "/statistik/")
                links[fase].append(f'https://www.transfermarkt.com.br{partida_link}')

        return links

## Coletando dados

### Ligas

In [7]:
comps = [('pl', 'premierleague'), ('lbr', 'brasileirao')]

p_catch = PartidaCatch()
c = Liga()

for comp in comps:
    seasons = Liga().get_competicoes_temporadas()[comp[0]]

    for ano in range(seasons[0], max(2020, seasons[1] - 1), -1):
        partidas = []

        try:
            links = c.partidas_estatistica_links(comp[0], ano)
        except:
            print(ano, 'deu errado!')
            continue
        
        for fase in links:
            print(comp[0], fase, ' | Ano:', ano)
            for i, jogo in enumerate(links[fase]):
                # print('Jogo:', i, jogo)
                try:
                    p_dict = p_catch.get_partida(jogo).get_partida_est()
                    p_dict['ano'], p_dict['fase'] = ano, fase
                    partidas.append(p_dict)
                except Exception as e:
                    # print(e, '-> Partida sem estatísticas!?')
                    continue
        
        os.makedirs("dados", exist_ok=True)
        os.makedirs(f"dados/{comp[1]}_partidas", exist_ok=True)
        
        pd.DataFrame(partidas).to_csv(f'dados/{comp[1]}_partidas/partidas_{ano}.csv', index=False)

pl 1.Rodada  | Ano: 2023
pl 2.Rodada  | Ano: 2023
pl 3.Rodada  | Ano: 2023
pl 4.Rodada  | Ano: 2023
pl 5.Rodada  | Ano: 2023
pl 6.Rodada  | Ano: 2023
pl 7.Rodada  | Ano: 2023
pl 8.Rodada  | Ano: 2023
pl 9.Rodada  | Ano: 2023
pl 10.Rodada  | Ano: 2023
pl 11.Rodada  | Ano: 2023
pl 12.Rodada  | Ano: 2023
pl 13.Rodada  | Ano: 2023
pl 14.Rodada  | Ano: 2023
pl 15.Rodada  | Ano: 2023
pl 16.Rodada  | Ano: 2023
pl 17.Rodada  | Ano: 2023
pl 18.Rodada  | Ano: 2023
pl 19.Rodada  | Ano: 2023
pl 20.Rodada  | Ano: 2023
pl 21.Rodada  | Ano: 2023
pl 22.Rodada  | Ano: 2023
pl 23.Rodada  | Ano: 2023
pl 24.Rodada  | Ano: 2023
pl 25.Rodada  | Ano: 2023
pl 26.Rodada  | Ano: 2023
pl 27.Rodada  | Ano: 2023
pl 28.Rodada  | Ano: 2023
pl 29.Rodada  | Ano: 2023
pl 30.Rodada  | Ano: 2023
pl 31.Rodada  | Ano: 2023
pl 32.Rodada  | Ano: 2023
pl 33.Rodada  | Ano: 2023
pl 34.Rodada  | Ano: 2023
pl 35.Rodada  | Ano: 2023
pl 36.Rodada  | Ano: 2023
pl 37.Rodada  | Ano: 2023
pl 1.Rodada  | Ano: 2022
pl 2.Rodada  | Ano: 20

### Copas

In [ ]:
comps = [('cbr', 'copabr'), ('la', 'liberta'), ('cl', 'champions')]

p_catch = PartidaCatch()
c = Copa()

for comp in comps:
    seasons = Copa().get_competicoes_temporadas()[comp[0]]

    for ano in range(seasons[0], max(2020, seasons[1] - 1), -1):
        partidas = []

        try:
            links = c.partidas_estatistica_links(comp[0], ano)
        except:
            print(ano, 'deu errado!')
            continue
        
        for fase in links:
            print(comp[0], fase, ' | Ano:', ano)
            for i, jogo in enumerate(links[fase]):
                print('Jogo:', i, jogo)
                try:
                    p_dict = p_catch.get_partida(jogo).get_partida_est()
                    p_dict['ano'], p_dict['fase'] = ano, fase
                    partidas.append(p_dict)
                except Exception as e:
                    print(e, '-> Partida sem estatísticas!?')
                    continue

        pd.DataFrame(partidas).to_csv(f'Dados/{comp[1]}_partidas/partidas_{ano}.csv', index=False)